In [1]:
import altair as alt
import pandas as pd

stateLevel = pd.read_csv("./data/2020_state_local.csv")
breakout = pd.read_csv("./data/2020_Breakout.csv")
states = alt.topo_feature('./data/us-10m.json', feature='states')


In [4]:
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

selection = alt.selection_single(empty='all', fields=['State'])

points = alt.Chart(stateLevel).mark_circle().encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    color=alt.condition(selection, alt.value('green'), alt.value('red')),
    size=alt.value(50),
    tooltip=[alt.Text('Full Name:N', title='State'), 'Total Spending:N']
).properties(
    title='Total Spending'
).add_selection(
    selection
)

spendBar = alt.Chart(breakout).transform_filter(
    selection
).mark_bar().encode(
    x=alt.X('sum(Spend_Mil):Q', title='Spending (millons)'),
    y=alt.Y('Function:N', title=''),
    color=alt.Color('Function:N', legend=None)
).properties(
    title='Spending Break Down',
        width=500
)

text = alt.Chart(breakout).transform_filter(
    selection
).mark_text(dx=20, dy=1, color='black').encode(
    x=alt.X('sum(Spend_Mil):Q'),
    y=alt.Y('Function:N', title=''),
    text=alt.Text('sum(Spend_Mil):Q', format=".1f")
)

(background + points & spendBar + text).save("FinalProject.html")
